# Warning
This file is currently a work in progress. Expect instability, inconsistency, and ugly code.
@W

In [1]:
DATA_DIRECTORY = "../../data"

# Delete me, eventually
data_path = DATA_DIRECTORY + '/dialog-babi-task1/'
training_file = data_path + 'dialog-babi-task1-API-calls-trn-workspace.txt'
dev_file = data_path + 'dialog-babi-task1-API-calls-dev-workspace.txt'

In [2]:
from functional import seq

In [3]:
import torch
import torch.nn.functional as F

In [4]:
# Initial object constructors
class Workspace():
    def __init__(self, tree):
        # assert tree is array
        self.tree = tree
        
    def add(address, content):
        assert isinstance(address, (str))
        assert isinstance(content, (str))
        
        def _add(tree):
            if not isinstance(tree, (list)):
                return tree
            elif tree[0] == address:
                return tree.concat([[content]])
            else:
                return map(tree, _add)
        
        return Workspace(_add(self.tree))

    def update(self, action):
        if action.what == 'INIT':
            return Workspace(['root'])
        elif action.what == 'MSG':
            return self
        elif action.what == 'ADD':
            tmp = action.content.split(' ')
            assert (len(tmp) == 2)
            address = tmp[0]
            address = tmp[1]
            return this.add(address, content)
        
    # TODO: Figure out how to flattenDeep    
    def to_words():
        return "NOT IMPLEMENTED YET"
    def to_tree():
        return this.tree
        

In [5]:
class Action():
    def __init__(self, who, what, content=False):
        self.who = who
        self.what = what
        self.content = content
        
    def to_words(self):
        return [f"{self.who}:", self.what] + (self.content.split(' ') if self.content else [])
        
    def to_string(self):
        if self.content:
            return f'{self.who}: {self.what} {self.content}'
        else:
            return f'{self.who}: {self.what}'

In [6]:
def files_to_words(filenames):
     return (seq(filenames)
                .map(lambda filename: open(filename).read().split())
                .flatten()
                .distinct()
                .sorted()
                .to_list())

In [7]:
def string_to_action(s):
    who = s[0]
    i = s.index(" ")
    line = s[i+1:]
    j = line.index(" ")
    what = line[0:j]
    content = line[j+1:]
    return Action(who, what, content)
    

In [8]:
#y = file_to_dialogs(training_file)

In [9]:
def file_to_dialogs(filename):
    init_entry = {
        "workspace": Workspace([]),
        "action": Action(who='A', what='INIT')
    }
    dialog_list = []
    current_workspace = init_entry["workspace"] 
    
    for dialog in open(filename).read().split('\n\n'):
        action_list = [init_entry]
        for action in dialog.split("\n"):
            action_list.append({"workspace": current_workspace,
                                "action": string_to_action(action)})
        dialog_list.append(action_list)
        
    return dialog_list
        
    

In [10]:
special_words = [
  '^', '$',            # start/end markers
  'INIT', 'WAIT',      # action types not in data
  'NO_OUTPUT',         # filler when copy or match word is not available
  'NO_USER_ACTION',    # initial value of lastUserAction
  'NO_WORKSPACE_WORD'
]

In [11]:
def load_babi_data():
    data_path = DATA_DIRECTORY + '/dialog-babi-task1/'
    training_file = data_path + 'dialog-babi-task1-API-calls-trn-workspace.txt'
    dev_file = data_path + 'dialog-babi-task1-API-calls-dev-workspace.txt'
    types_and_tokens = [
            { "type": 'cuisine', "tokens": ['french', 'italian', 'british', 'spanish', 'indian'] },
            { "type": 'location', "tokens": ['rome', 'london', 'bombay', 'paris', 'madrid'] },
            { "type": 'price', "tokens": ['cheap', 'moderate', 'expensive'] },
            { "type": 'people', "tokens": ['two', 'four', 'six', 'eight'] },
        ]
    
    data = {
      "training": file_to_dialogs(training_file),
      "dev": file_to_dialogs(dev_file),
      "words": files_to_words([training_file]) + special_words,
      "types_and_tokens": types_and_tokens
    }

    return data

In [12]:
def create_one_hot(vocab_size):
    return ([[int(i == j) for i in range(vocab_size)] for j in range(vocab_size)])


In [13]:
def action_to_one_hot(action):
    return ((seq(['^'] + action.to_words() + ['$'])
                .map(lambda w: data['words'].index(w))
                .map(lambda i: one_hot[i])
                .to_list()))

In [14]:
def action_to_idx(action):
    return ((seq(['^'] + action.to_words() + ['$'])
                .map(lambda w: data['words'].index(w))
                .to_list()))

In [15]:
MAX_TENSOR_SIZE = 23 #TODO: Refactor to determine automatically
def pad_tensor(tensor):
    m = MAX_TENSOR_SIZE
    a = tensor
    return a + [([0 for i in range(vocab_size)]) for j in range(m - len(a))]

In [16]:
MAX_TENSOR_SIZE = 23 #TODO: Refactor to determine automatically
def pad_word_tensor(tensor):
    m = MAX_TENSOR_SIZE
    a = tensor
    return a + [vocab_size-1 for j in range(m - len(a))]

In [17]:
def actions_to_one_hot(data):
    new_data = (seq(data)
                .map(lambda a: a['action'])
                .map(lambda a: action_to_one_hot(a))
                .map(pad_tensor)
                .map(torch.LongTensor))
    return new_data

                #.map(action_to_one_hot))

In [18]:
def actions_to_idx(data):
    new_data = (seq(data)
                .map(lambda a: a['action'])
                .filter(lambda a: a.what == "MSG")
                .map(lambda a: action_to_idx(a)))
                #.map(pad_word_tensor)
                #.map(torch.LongTensor))
    return new_data

                #.map(action_to_one_hot))

In [19]:
def dialogs_to_one_hot(dialogs):
    out_data = []
    for dialog in dialogs:
        out = torch.stack(list(actions_to_one_hot(dialog)))
        out_data.append(out)
    return out_data

In [20]:
def dialogs_to_idx(dialogs):
    out_data = []
    for dialog in dialogs:
       # out = torch.stack(list(actions_to_idx(dialog)))
        out = list(actions_to_idx(dialog))
        out_data.append(out)
    return out_data

In [21]:
data = load_babi_data()

In [22]:
data['training'][1][-1]['action'].to_string()

'A: MSG api_call italian london eight cheap'

In [23]:
vocab_size = len(data["words"])
#one_hot = create_one_hot(vocab_size)

In [24]:
# Convert our data into a list of s

#training_data = dialogs_to_one_hot(data['training'])
training_data = (dialogs_to_idx(data['training']))



In [26]:
def idx_to_words(idxs): return list(map(lambda i: data['words'][int(i)], idxs))

In [27]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [28]:
# Adapted from:
# http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

class Encoder(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.h_size = h_size
        self.x_size = x_size
        self.cell = nn.GRU(h_size, h_size)
        self.embedding = nn.Embedding(x_size, h_size)
        
        #self.embedding = nn.Embedding(x_size, h_size)

    # x: input
    # h: hidden state
    def forward(self, input, h):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        for i in range(self.n_layers):
            output, h = self.cell(output, h)
        return output, h
    
    def init_hidden(self):
        return Variable(torch.zeros(1, 1, self.h_size))

In [29]:
class DecoderMock(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        super(DecoderMock, self).__init__()
        self.n_layers = n_layers
        self.h_size = h_size
        self.x_size = x_size
        self.cell = nn.GRU(h_size, h_size)
        self.out = nn.Linear(h_size, x_size)
        self.softmax = nn.LogSoftmax()
        self.embedding = nn.Embedding(x_size, h_size)

    # x: input
    # h: hidden state
    def forward(self, input, h):
        output = self.embedding(input).view(1, 1, -1)
        for i in range(self.n_layers):
            output = F.relu(output)
            output, h = self.cell(output, h)
        output = self.softmax(self.out(output[0]))
        return output, h
    
    def init_hidden(self):
        return Variable(torch.zeros(1, 1, self.h_size))

In [30]:
# TODO: Implement initialization

# Whether this is a good idea is still in question
attentional_net = torch.nn.Sequential(
                    torch.nn.Linear(2, 1),
                    torch.nn.Tanh())

# Whether this is a good idea is still in question
decoder_output_net = torch.nn.Sequential(
                        torch.nn.Softmax())

class Decoder(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.h_size = h_size
        self.cell = nn.GRU(x_size, h_size)
        self.embedding = nn.Embedding(x_size, h_size)

        self.decoder_out = torch.nn.Sequential(torch.nn.Linear(h_size + x_size + h_size, x_size),
                                               torch.nn.LogSoftmax())
        self.attention = torch.nn.Sequential(torch.nn.Linear(2 * x_size, 1),
                                             torch.nn.Tanh())

    def forward(self, decoder_state, encoder_states):
        
        embed = self.embedding(decoder_state).view(1, 1, -1)
        decoder_state = embed
        
        # Combine State
        combinedState = lambda encoder_state, z: self.attention(torch.cat(decoder_state, encoder_state), 0)

        # This is terrible and I feel dirty
        zero_states = torch.zeros(encoder_states.size()[0])

        # Create attention vector
        attention_vector = F.softmax(torch.Tensor.map_(encoder_states.data, zero_states.data,
                                                       combinedState))

        
        final = encoder_states * torch.transpose(attention_vector.unsqueeze(0), 0, 1).expand(encoder_states.size())

        for i in range(self.n_layers):
            decoder_state, h = self.cell(decoder_state, h)
        decoder_state = self.decoder_out(torch.concat(decoder_state, embed, attention_vector))
        return decoder_state, h

In [31]:
 class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()
        
    def forward(self, input, hidden, encoder_states):
        output = self.embedding(input).view(1, 1, -1)
        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden, attention

    def initHidden(self):
        return Variable(torch.zeros(1, 1, self.hidden_size))

One day, we will be a neural network

In [33]:
# Fix for CUDNN memory leak 
# https://discuss.pytorch.org/t/tracking-down-a-suspected-memory-leak/1130/39
#torch.backends.cudnn.enabled = False


# Init Training here

In [34]:
hidden_size = 256
learning_rate = 0.01

In [35]:
e = 1000
encodern = Encoder(vocab_size, hidden_size)
decodern = DecoderMock(vocab_size, hidden_size)
#decodern = Decoder(vocab_size, hidden_size)
plot_losses = []
plot_every = 1
plot_loss_total = 0

encoder_optimizer = torch.optim.SGD(encodern.parameters(), lr=learning_rate)
decoder_optimizer = torch.optim.SGD(decodern.parameters(), lr=learning_rate)
criterion = nn.NLLLoss() 
loss = 0

losses = []


In [36]:
from time import gmtime, strftime
get_time = lambda: strftime("%Y-%m-%d %H:%M:%S", gmtime())

In [37]:
def chunk_by(in_list, by): return (seq(in_list).grouped(by).map(lambda l: list(l)).to_list())

In [38]:
def train_epochs(e=100, batch_size=10):
    for ep, epoch in enumerate(range(e)):
        print("Epoch:" +  str(ep))
        torch.save(encodern, open("encoder" + get_time() + "-epoch-" + str(ep),'wb'))
        torch.save(decodern, open("decoder" + get_time() + "-epoch-" + str(ep),'wb'))
        plot_loss_total = 0 
        #for i in range(batch)
        encoder_hidden = encodern.init_hidden()
        decoder_hidden = decodern.init_hidden()

            
        for i, batch in enumerate(chunk_by(training_data, batch_size)):
            #print("Batch: " + str(i))
            loss = train(batch, encodern, decodern, encoder_optimizer, decoder_optimizer, encoder_hidden, decoder_hidden, criterion)
            plot_loss_total += loss[0]

            if epoch % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0 
            print(plot_loss_avg)
        torch.save(encodern, open("encoder" + get_time() + "-epoch-" + str(ep),'wb'))
        torch.save(decodern, open("decoder" + get_time() + "-epoch-" + str(ep),'wb'))
    #showPlot(plot_losses)

In [39]:
import random
import gc

In [43]:
def train(dialog_data, encodern, decodern, encoder_optimizer, decoder_optimizer, encoder_hidden, decoder_hidden, criterion):
   
    encodern.zero_grad()
    decodern.zero_grad()


    encoder_outputs = Variable(torch.zeros(1, encodern.h_size))

    sos = data['words'].index('^')
    eos = data['words'].index('$')

    encoder_input = Variable(torch.LongTensor([sos]))
    decoder_input = Variable(torch.LongTensor([sos]))

    #encoder_input = Variable(torch.LongTensor([word]))

    loss = 0

    #training_variables = list(map(lambda d: Variable(d), training_data))

    # This may require a little explanation
    # we use history as a cache of messages
    # at every "action" in the dialogs we attempt
    # to predict it given the previously given history
    for it, dialog in enumerate(dialog_data): 
        gc.collect()
        print("DIALOG: " + str(it))

        # I'm not sure of a better way to optimize this, due to our sequences being variable length
        history = []
        for action in dialog:

            # Append our action to our history
            history = history + [action]

            # If we have only one action we can't have both training and test
            if len(history) < 2: continue

            # Iterate thruugh every message(action) in the history other than the current
            for message in history[:-1]:
                for word in message:
                    encoder_word = word
                    encoder_input = Variable(torch.LongTensor([word]))
                    encoder_outputs, encoder_hidden = encodern(encoder_input, encoder_hidden)

            decoder_hidden = encoder_hidden # Share our hidden state

            # Seed our decoder with the start of sentence token

            # Start Decoding Cycle
            # Note that we use its own predictions as the next input
                # Q: Should I add teacher forcing?
            int_d_outputs = []
            for line in history[-1]:
                decoder_output, decoder_hidden =  decodern(decoder_input, decoder_hidden)
            
                topv, topi = decoder_output.data.topk(1)
                ni = topi[0][0]
                if random.random() > 0.9:
                    decoder_input = Variable(torch.LongTensor([[ni]]))
                else:
                    decoder_input = Variable(torch.LongTensor([[line]]))
 
                int_d_outputs.append(data['words'][ni])
                loss += criterion(decoder_output[-1],
                                  Variable(torch.LongTensor([line])))
            
                if ni == eos:
                    break    
                    
            
    loss.backward()
     
    encoder_optimizer.step()
    decoder_optimizer.step() 
    return loss


In [42]:
# Train our model!
train_epochs()

Epoch:0


/usr/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type DecoderMock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


DIALOG: 0
DIALOG: 1
DIALOG: 2
DIALOG: 3
DIALOG: 4
DIALOG: 5
DIALOG: 6
DIALOG: 7
DIALOG: 8
DIALOG: 9


KeyboardInterrupt: 

# WARNING!
Everything below this cell is currently non-functional. If it compiles, I'll be rather surprised.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2) # this locator puts ticks at regular intervals
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    

In [ ]:
criterion(Variable(a, requires_grad=True),b)

In [ ]:
# TODO: Implement initialization

# Whether this is a good idea is still in question
attentional_net = torch.nn.Sequential(
                    torch.nn.Linear(2, 1),
                    torch.nn.Tanh())

# Whether this is a good idea is still in question
decoder_output_net = torch.nn.Sequential(
                        torch.nn.Softmax())

class Decoder(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        self.n_layers = n_layers
        self.h_size = h_size
        self.cell = nn.GRU(x_size, h_size)

        self.out = nn.Linear(h_size, )
        self.attention = torch.nn.Sequential(torch.nn.Linear(2 * x_size, 1),
                                             torch.nn.Tanh())

    def forward(self, decoder_state, encoder_states):
        # Combine State
        combinedState = lambda encoder_state, z: self.attention(torch.cat(decoder_state, encoder_state), 0)

        # This is terrible and I feel dirty
        zero_states = torch.zeros(encoder_states.size()[0])

        # Create attention vector
        attention_vector = T.softmax(torch.Tensor.map_(encoder_states, zero_states,
                                                       combinedState))

        final = encoder_states * torch.transpose(attention_vector.unsqueeze(0), 0, 1).expand(encoder_states.size())

        for i in range(self.n_layers):
            output, h = self.cell(output, h)
        return output, h

In [ ]:
def go(e):
    encodern = Encoder(vocab_size, hidden_size). 
    decodern = DecoderMock(vocab_size, hidden_size). 
    plot_losses = []
    plot_every = 1
    plot_loss_total = 0
    
    encoder_optimizer = torch.optim.SGD(encodern.parameters(), lr=learning_rate)
    decoder_optimizer = torch.optim.SGD(decodern.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss() 
    loss = 0
    
    losses = []
    for epoch in range(e):
        loss = build_and_train(encodern, decodern, encoder_optimizer, decoder_optimizer, criterion)
        plot_loss_total += loss[0]
        
        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0 
        print(plot_loss_avg)
    #showPlot(plot_losses)
    
 

In [ ]:
class DecoderMock(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        super(DecoderMock, self).__init__()
        self.n_layers = n_layers
        self.h_size = h_size
        self.x_size = x_size
        self.cell = nn.GRU(h_size, h_size)
        self.out = nn.Linear(h_size, x_size)
        self.softmax = nn.LogSoftmax()
        self.attention = torch.nn.Sequential(torch.nn.Linear(2 * x_size, 1),
                                             torch.nn.Tanh())

        self.embedding = nn.Embedding(x_size, h_size)

    # x: input
    # h: hidden state
    def forward(self, input, h, encoder_states):
        
        embedded = self.embedding(input).view(1, 1, -1)
        print(encoder_states)
        print(h)
        combinedState = lambda encoder_state: self.attention(torch.cat((h, encoder_state), 1))
        attention_vector = torch.cat(h,encoder_states)
        
        output = self.embedding(input).view(1, 1, -1)
        for i in range(self.n_layers):
            output = F.relu(output)
            output, h = self.cell(output, h)
        output = self.softmax(self.out(output[0]))
        return output, h, attention
    
    def init_hidden(self):
        return Variable(torch.zeros(1, 1, self.h_size))